# CERTAMEN 1, ANÁLISIS NÚMERICO II

Daniela Paz Díaz Mora 

201710003-6

In [1]:
import numpy as np
import numpy.linalg as alg
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import seaborn as sns
import scipy.stats as sp
import math
from math import copysign, hypot
from scipy.misc import derivative
from scipy.linalg import lu, inv
import scipy

### **5**.

In [2]:
m=50
n=12
t=np.linspace(0,1,50)
print('el vector t corresponde a \n ', t)
A=np.vander(t,12,increasing=True)[0:50]
print('la matriz A de dim mxn corresponde a \n', A,'\n',A.shape)
b=np.cos(t*4)
print('el vector b  corresponde a \n', b,'\n',b.shape)

el vector t corresponde a 
  [0.         0.02040816 0.04081633 0.06122449 0.08163265 0.10204082
 0.12244898 0.14285714 0.16326531 0.18367347 0.20408163 0.2244898
 0.24489796 0.26530612 0.28571429 0.30612245 0.32653061 0.34693878
 0.36734694 0.3877551  0.40816327 0.42857143 0.44897959 0.46938776
 0.48979592 0.51020408 0.53061224 0.55102041 0.57142857 0.59183673
 0.6122449  0.63265306 0.65306122 0.67346939 0.69387755 0.71428571
 0.73469388 0.75510204 0.7755102  0.79591837 0.81632653 0.83673469
 0.85714286 0.87755102 0.89795918 0.91836735 0.93877551 0.95918367
 0.97959184 1.        ]
la matriz A de dim mxn corresponde a 
 [[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 2.04081633e-02 4.16493128e-04 8.49985975e-06
  1.73466526e-07 3.54013317e-09 7.22476158e-11 1.47444114e-12
  3.00906355e-14 6.14094602e-16 1.25325429e-17 2.55766182e-19]

### a. Se tiene que hay que encontrar Ax=b, tal que x sea minimo

In [40]:
ra=scipy.linalg.lstsq(A,b)[0]
np.set_printoptions(precision=16,suppress=True)                
print(ra)

[ 1.0000000009966121 -0.000000422743446  -7.999981235675575
 -0.0003187633453632 10.66943079660686   -0.0138202907076439
 -5.647075620612139  -0.0753160353494552  1.6936069753360874
  0.0060321007054991 -0.3742417003099015  0.0880405755367036]


### b. Factorización QR calculada por algoritmo Gram-Schmidt

- Tenemos que x=R^{-1}Q^Hb, QR=A

In [4]:
def mgs(matrix: np.array) -> (np.array, np.array):
    
    q1 = np.array(matrix, dtype='float64')
    r1 = np.zeros((n, n))
    for k in range(n):
        a_k = q1[..., k]
        r1[k,k] = np.linalg.norm(a_k)
        a_k /= r1[k, k]
        for i in range(k+1, n):
            a_i = q1[..., i]
            r1[k,i] = np.transpose(a_k) @ a_i
            a_i -= r1[k, i] * a_k
    return q1, r1

In [5]:
Qb,Rb=mgs(A)
#np.linalg.inv(Rb)*Qb.T
rb=np.dot(np.dot(np.linalg.inv(Rb),Qb.T),b)
print(rb)

[ 0.9999999974886997  0.0000006109656248 -8.000020069315525
  0.0002536139254516 10.665047574730124   0.0059659525577445
 -5.7031891493243165  0.0274716310668737  1.5721322300378233
  0.0954681739676744 -0.4115481095213909  0.0947739273688057]


### c. Factorización QR calculada por algoritmo triangulación de HouseHolder

- Tenemos que x=R^{-1}Q^Hb, QR=A

In [6]:
def  house(A): 
    ident = np.eye(m)
    W = np.zeros((m,n)) #matrix of vectors
    R = np.copy(A)

    for k in range(0,n):
        e1=ident[k:m,k]
        x = R[k:m,k]
        W[k:m,k]= ( int(x[0]>=0) - int(x[0]<0) )*np.linalg.norm(x,2)*e1+x
        W[k:m,k]=W[k:m,k]/np.linalg.norm(W[k:m,k],2)
        R[k:m][:,k:n]=R[k:m][:,k:n]-np.dot(np.outer(2*W[k:m,k],W[k:m,k]),W[k:m][:,k:n])


    Q=np.zeros((m,m))

    #QI = Q, column by column
    for i in range(0,m):
        Q[i,i]=1
        for k in range(n-1,-1,-1): 
            Q[k:m,i]=Q[k:m,i]-2*np.dot(np.outer(W[k:m,k],W[k:m,k]),Q[k:m,i])

    return Q,R


In [12]:
Qc,Rc=house(A)
#print(Qc.shape,Rc.shape,b.shape)
rc=np.dot(np.dot(scipy.linalg.pinv(Rc),Qc),b)
print(rc)

[ 0.2539532956300719 -0.770726362504823  -0.5051129756402695
 -0.2299915917638745 -0.0729509888461815  0.0012717043992745
  0.0282765595426827  0.0307598871818028  0.0220123435629932
  0.0096048500285067 -0.0022471537249328 -1.131746403884529 ]


claramene no me dio :(

### d. Factorización QR 

In [8]:
Qd,Rd= alg.qr(A)
rd=np.dot(np.dot(np.linalg.inv(Rd),Qd.T),b)
print(rd)

[ 1.0000000009966112 -0.0000004227434149 -7.999981235675847
 -0.0003187633401467 10.669430796609959  -0.0138202905509388
 -5.647075620596297  -0.0753160355379805  1.6936069753719494
  0.0060320997145027 -0.3742417001049034  0.088040575508785 ]


### e. x=A\b

Como se tiene que no se le puede calcular la inversa de A, se utilizara la pseudoinvera para calcular x=A\b

In [13]:
inv_A=np.linalg.pinv(A)
re=np.dot(inv_A,b)
print(re)

[ 1.0000000009966115 -0.0000004227433998 -7.99998123567525
 -0.0003187633451489 10.669430796631787  -0.0138202906236984
 -5.647075620130636  -0.0753160348394886  1.693606975954026
  0.0060321007622406 -0.3742416999302804  0.0880405754724052]


### f. Factorización SVD

x=V\Sigma^{-19

In [10]:
U,S,Vh=scipy.linalg.svd(A)
Sigma=np.append(np.diag(S),np.zeros((m-n,n)),axis=0)
#print(U.T.shape,b.shape)
f1=np.dot(U.T,b)
#print(f1.shape)
#print(Vh.T.shape,np.linalg.pinv(Sigma).shape)
f2=np.dot(Vh.T,np.linalg.pinv(Sigma))
rf=np.dot(f2,f1)
print(rf)

[ 1.0000000009966108 -0.0000004227434132 -7.999981235676155
 -0.0003187633399191 10.669430796577236  -0.0138202906080331
 -5.647075620825223  -0.0753160350600348  1.6936069750958254
  0.0060321008151971 -0.3742417003296958  0.0880405755358424]


In [17]:
np.linalg.svd(np.matrix([[1,1],[0,0]]))

(matrix([[0.9999999999999999, 0.                ],
         [0.                , 0.9999999999999999]]),
 array([1.4142135623730951, 0.                ]),
 matrix([[ 0.7071067811865475,  0.7071067811865475],
         [-0.7071067811865475,  0.7071067811865475]]))

### Diferencia de decimales

In [71]:
#a y b
np.sum(ra-rb)

-3.5093503103489354e-09

In [72]:
#a y c
np.sum(ra-rc)

1.7132532164575174

In [73]:
#a y d
np.sum(ra-rd)

7.859606854232482e-10

In [74]:
#a y e
np.sum(ra-re)

-2.0909284859094157e-09

In [75]:
#a y f
np.sum(ra-rf)

1.609823385706477e-15

Podemos ver que la aproximación más estable es cuando se uso SVD reducida (f) y la más inestable (obviamente obviando el resultado de c) es cuando se hizo mediante GS modificado